# 使用tensorflow

tensorflow是一个强大的用于数值计算的开源软件，尤其适用于大型机器学习。

In [47]:
# 准备工作
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import numpy as np
import os

# 控制随机种子，使结果一样
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = ".."
CHAPTER_ID = "tensorflow"


def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# 创建和运行数据流图

In [48]:
import tensorflow as tf

reset_graph()
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

In [49]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [50]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
result

42

In [51]:
sess.close()

In [52]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval() #eval:串演算指令

In [53]:
result

42

In [54]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [55]:
result

42

In [56]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()
result = f.eval()
result

/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


42

In [57]:
sess.close()